In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
## Processing function for removing smart quotes

def remove_smartquotes(frame):
    frame_ = frame.copy()
    frame_.loc[:,['message']] = frame_.message.apply(lambda x: x.replace("‘", "'"))
    frame_.loc[:,['message']] = frame_.message.apply(lambda x: x.replace("’", "'"))
    frame_.loc[:,['message']] = frame_.message.apply(lambda x: x.replace("“", "\""))
    frame_.loc[:,['message']] = frame_.message.apply(lambda x: x.replace("”", "\""))
    return frame_

In [3]:
## More processing code that will convert the slack channel csv into something usable

def process_slack_channel(filename):
    #Processing raw channel messages
    channel_frame = pd.read_csv(filename, header=None)
    print("total number of messages:", len(channel_frame))
    
    channel_frame.columns = ['year', 'channel_id', 'user_id', 'message', 'timestamp']
    channel_frame.message = channel_frame.message.apply(lambda x:x.strip('"'))

    #Throw out user action notifications, re-index
    channel_frame = channel_frame[channel_frame.message.apply(lambda x: re.match(r'\\u.+\|.+\\u003e', x) is None)]
    channel_frame.index = range(len(channel_frame))
    print("after throwing out channel events:", len(channel_frame))
    



    #Throw out empty lines, reindex
    channel_frame = channel_frame[channel_frame.message != '']
    channel_frame.index = range(len(channel_frame))

    channel_frame.message = channel_frame.message.apply(lambda x: x.lower())
    print("after throwing out empty messages:", len(channel_frame))
    
    #Replace users with |USER| tag
    channel_frame.message = channel_frame.message.apply(lambda x: re.sub(r"\\u003.+?\\u003e", "|USER|", x))
    
    #Convert timestamp to actual timestamp object
    
    channel_frame.timestamp = pd.to_datetime(channel_frame.timestamp, unit='s')

    
    return remove_smartquotes(channel_frame)

In [5]:
## Put the filename in the quotes below

support_messages = process_slack_channel('support_messages.csv')

total number of messages: 48558
after throwing out channel events: 45240
after throwing out empty messages: 45185


In [9]:
support_messages # uncomment to show messages

,year,channel_id,user_id,message,timestamp
0,2011,C0DUNQVPS,U0BPUAG7N,sounds good,2015-11-18 23:44:09.000008
1,2011,C0ELD9Y3G,U0BTA8US0,fyi i just added a new knowledge article to sc...,2016-07-30 02:19:27.000002
2,2011,C0ELD9Y3G,U02V8GURN,perfect thank you.,2015-11-17 16:40:05.000004
3,2011,C0ELD9Y3G,U0BSVEW4Q,yes. they will be updating the queue spreadsh...,2015-11-17 16:39:47.000003
4,2011,C0ELD9Y3G,U02V8GURN,hi lisa - just needing a little clarification ...,2015-11-17 16:38:41.000002
5,2011,C0ELD9Y3G,U0BSVEW4Q,"one note. for a few of the entries, you indic...",2015-11-17 15:05:54.000007
6,2011,C0ELD9Y3G,U0BSVEW4Q,"thanks, amy.",2015-11-17 15:02:01.000006
7,2011,C0ELD9Y3G,U0DN4DLH4,i can look at more tomorrow,2015-11-17 05:23:20.000005
8,2011,C0ELD9Y3G,U0DN4DLH4,"no different than they would need to do, i've...",2015-11-17 05:23:15.000004
9,2011,C0ELD9Y3G,U0DN4DLH4,"so i've been through lines 6-41, most were in...",2015-11-17 05:21:52.000003


In [6]:
## Here you need to generate your contexts.
## Depending on what your particular assignment is, you need to group the text in different ways.
## You should end up with a contexts that is a list of strings

contexts = []

for i in foo:
    contexts.append(bar)

NameError: name 'foo' is not defined

In [1]:
## In this cell you need to 
## A) initialize your vectorizer object with the settings you will want to use to make
## your bag of words vectors
## B) Fit your vectorizer and generate a matrix of vectors

vectorizer = some_vectorizer_initialization_function
tfidf_vectors = vectorizer.fit_transform(some_kind_of_input)

NameError: name 'some_vectorizer_initialization_function' is not defined

array([13276, 17513,  2564, ...,  6753,  6754,  9284])

In [115]:
def get_similar_terms(term):
    term_vector = bow_vector_for_the_term
    
    cosine_similarities = cosine_similarities_between_term_and_all_other_terms
    
    
    ## This is a way of making sure the terms distances are sorted in the corect order, so we take the
    ## top 10.
    top_10 = np.flip(np.argsort(cosine_similarities, axis=0), axis=0).T[0][:10]
    results = []
    for i in top_10:
        results.append(vectorizer.get_feature_names()[i])
    return results
        

In [2]:
get_similar_terms('printer')

NameError: name 'get_similar_terms' is not defined